In [2]:
from sqlalchemy import create_engine, inspect
import creds
import pandas as pd

In [3]:
host = creds.host
password = creds.password
port = creds.port
user = creds.user
database = creds.database
api_type = creds.api_type
engine = create_engine(f'postgresql+{api_type}://{user}:{password}@{host}:{port}/{database}')
# engine.connect()

In [4]:
df = pd.read_sql_table('products', engine)

In [86]:
### Cleaning up the price column in the dataframe.
# Removes all rows which have "N/A" in the price column.
df = df[df['price'] != 'N/A']
# Removes all '£' from the price column
df['price'] = df['price'].str.strip('£')
df['price'] = df['price'].str.replace(',', '')
df['price'] = df['price'].astype('float64')
df.head(100)
df.dtypes

,id,product_name,category,product_description,price,location,page_id,create_time
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",5.00,"Wokingham, Berkshire",1426704584,2022-02-26
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,20.00,"Inverness, Highland",1426704579,2022-02-26
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,20.00,"Skegness, Lincolnshire",1426704576,2022-02-26
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,115.00,"Radstock, Somerset",1426704575,2022-02-26
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,450.00,"Delph, Manchester",1426704570,2022-02-26
...,...,...,...,...,...,...,...,...
103,248ba22c-0329-4132-aa33-0036b35d9884,"3 Seater Sofa | in Carshalton, London | Gumtree","Home & Garden / Dining, Living Room Furniture ...","3 Seater Leather Sofa - Used, but good conditi...",100.00,"Carshalton, London",1426701637,2022-02-26
104,e0fc0443-7cea-43a9-8446-f3c0ea7fd2d9,"Big Taxi canvas picture | in Kirkcaldy, Fife |...","Home & Garden / Dining, Living Room Furniture ...",Big Taxi canvas picture\ras good as new \rDime...,5.00,"Kirkcaldy, Fife",1426701627,2022-02-26
105,ca4f0b85-219e-44e3-8e19-7b4425a2ffc4,"Family of 4 wooden ducks | in St Andrews, Fife...",Home & Garden / Other Household Goods,Family of 4 wooden ducks poor things have been...,45.00,"St Andrews, Fife",1420504407,2022-02-26
107,d3423b1d-2565-416a-935e-056d67363c12,"Farmhouse table | in Telscombe Cliffs, East Su...","Home & Garden / Dining, Living Room Furniture ...",Farmhouse Table \r160cm x 80cm \rseats 6 comfo...,25.00,"Telscombe Cliffs, East Sussex",1426701614,2022-02-26


In [1]:
df['category'] = df['category'].astype('category')
df['category']

NameError: name 'df' is not defined